### 導入模板庫

In [ ]:
import dicom
import numpy as np
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
from PIL import Image
import shutil
import scipy.misc
import random
from matplotlib import pyplot as plt
import SimpleITK as sitk
import json
from torchvision import transforms as T
from torchvision.transforms import functional as F

def rm_mkdir(dir_path):
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
        print('Remove path - %s'%dir_path)
    os.makedirs(dir_path)
    print('Create path - %s'%dir_path)

### Examples demo

In [ ]:
dcm = dicom.read_file('./ISIC/dataset/cancer/B题-全部数据/数据集1/1001/arterial phase/10001.dcm')
dcm

In [ ]:
image = sitk.ReadImage('./ISIC/dataset/cancer/B题-全部数据/数据集1/1001/arterial phase/10001.dcm')
image_array = sitk.GetArrayFromImage(image)
plt.imshow(convertImage(image_array[0,:,:]), 'gray')

In [ ]:
# import SimpleITK as sitk
# import numpy as np
# image = sitk.ReadImage('./ISIC/dataset/cancer/B题-全部数据/数据集1/1001/arterial phase/10009.dcm')
# image_array = sitk.GetArrayFromImage(image)
# dcm.image = image_array[0,:,:]
# convertImg = convertImage(dcm.image)
# plt.imshow(convertImg, 'gray')
# imgtmp = np.transpose(image_array, axes=(1,2,0))
# imgtmp.shape
# plt.imshow(imgtmp[200:475,125:375,0], 'gray')
# plt.imshow(image_array[0,:,:], 'gray')

In [ ]:
tmp_raw_mask = Image.open('./ISIC/dataset/cancer/B题-全部数据/数据集1/1100/venous phase/20011_mask.png')
#tmp_raw_mask = np.array(tmp_raw_mask)
plt.imshow(tmp_raw_mask, 'gray')
plt.savefig('原始掩模.png')

In [ ]:
tmp_raw_mask.save('sss.png')

In [ ]:
#tmp_cut_mask = convertImage(tmp_raw_mask*255)
plt.imshow(tmp_raw_mask[Loc[0]:Loc[1], Loc[2]:Loc[3]], 'gray')
plt.savefig('裁剪掩模.png')

In [ ]:
#dcm = dicom.read_file('./ISIC/dataset/cancer/B题-全部数据/数据集1/1100/venous phase/20029.dcm')
image = sitk.ReadImage('./ISIC/dataset/cancer/B题-全部数据/数据集1/1100/venous phase/20011.dcm')
image_array = sitk.GetArrayFromImage(image)
plt.imshow((image_array[0, :, :]), 'gray')
plt.savefig('原始图像.png')

In [ ]:
plt.imshow(convertImage(image_array[0, :, :]), 'gray')
plt.savefig('图像增强.png')
a = convertImage(image_array[0, :, :])

In [ ]:
plt.imshow((a[Loc[0]:Loc[1], Loc[2]:Loc[3]]), 'gray')
plt.savefig('图像裁剪.png')


In [ ]:
tmp_PIL = Image.fromarray(np.uint8(a[Loc[0]:Loc[1], Loc[2]:Loc[3]]*255.0))

In [ ]:
def show_augmentation(raw):
        
    image = raw 
    aspect_ratio = image.size[1]/image.size[0]
    hw = (image.size[0],image.size[1])
    print('圖像尺寸 : ', image.size[0],image.size[1])
    Transform = []
    ### 縮放到 300*X 或者 X*300 或者 X*320 或者 320*X
    ResizeRange = random.randint(300,320)
    Transform.append(T.Resize((int(ResizeRange*aspect_ratio),ResizeRange)))
    p_transform = random.random()
    
    ### 旋轉 0,90,180,270°
#     RotationDegrees = [0,90,180,270]
#     RotationDegree = random.randint(0,3)
#     RotationDegree = RotationDegrees[RotationDegree]
#     print('旋轉 : ', RotationDegree)
#     Transform.append(T.RandomRotation((RotationDegree,RotationDegree)))
    
   
    
    ### 方向調整 -10°~10°
    RotationRange = random.randint(-10,10)
    print('方向調整 : ', RotationRange)
    Transform.append(T.RandomRotation((RotationRange,RotationRange)))
    
    sTransform = T.Compose(Transform)
    #tmp = sTransform(image) 
    
    ### 按照中心裁剪 
    CropRange = random.randint(250,270)
    Transform.append(T.CenterCrop((int(CropRange*aspect_ratio),CropRange)))
    print('按照中心裁剪 : ', CropRange*aspect_ratio,CropRange)
    
    
    Transform = T.Compose(Transform)

    image = Transform(image)
    #tmp = image
        
    ### 平移裁剪
    ShiftRange_left = random.randint(0,20)
    ShiftRange_upper = random.randint(0,20)
    ShiftRange_right = image.size[0] - random.randint(0,20)
    ShiftRange_lower = image.size[1] - random.randint(0,20)
    image = image.crop(box=(ShiftRange_left,ShiftRange_upper,ShiftRange_right,ShiftRange_lower))
    print('平移變換(左上右下)', ShiftRange_left, ShiftRange_upper, ShiftRange_right, ShiftRange_lower)
    
    ### 水平翻轉
    if random.random() < 0.5:
        print('水平翻轉')
        image = F.hflip(image)
    tmp = image
    ### 垂直翻轉
#     if random.random() < 0.5:
#         print('垂直翻轉')
#         image = F.vflip(image) 
    Transform = []
    
    Transform = T.ColorJitter(brightness=0.2,contrast=0.2,hue=0.02)
    
    image = Transform(image)
    tmp = image
    ### 縮放到 256*X 或 X*256
    Transform = []
    Transform.append(T.Resize((int(256*aspect_ratio)-int(256*aspect_ratio)%16,256)))
    
    Transform = T.Compose(Transform)
    transImg = Transform(image)
    
    return transImg, tmp 
#img = Image.open('./B题示例数据/CT影像/1002/arterial phase/10076_contour.png')
TransImg, tmpImg = show_augmentation(tmp_PIL)
tmpImg
#tmpImg.save
#TransImg

In [ ]:
tmpImg.save('對比度調整.png')

### 提取並裁剪數據 
 考慮到直腸癌發病區域較爲固定， 而其餘區域可以認爲是‘背景’， 故我們對輪廓圖片和mask圖片進行切割 切割範圍爲（170:490， 100:420）

In [ ]:
### 數據路徑及分配路徑
#Paths = './B题示例数据/CT影像/'
Paths = './ISIC/dataset/cancer/B题-全部数据/数据集1/'
list_Paths = os.listdir(Paths)
Dataset = './ISIC/dataset/cancer/Input_all/'
Dataset_mask = './ISIC/dataset/cancer/Mask_all/'

In [ ]:
### 裁剪區域
#Loc = (210, 466, 125, 381)
Loc = (170, 490, 100, 420)
# 230-430 189-321
def data_augmentation(raw_path, raw_gt_path):
    image = Image.open(raw_path)
    GT = Image.open(raw_gt_path).convert('L')
    
    aspect_ratio = image.size[1]/image.size[0]
    hw = (image.size[0],image.size[1])
    Transform = []

    ResizeRange = random.randint(300,320)
    Transform.append(T.Resize((int(ResizeRange*aspect_ratio),ResizeRange)))
    p_transform = random.random()

    if p_transform <= 0.6:
        
        RotationRange = random.randint(-15,15)
        Transform.append(T.RandomRotation((RotationRange,RotationRange)))
        CropRange = random.randint(250,270)
        Transform.append(T.CenterCrop((int(CropRange*aspect_ratio),CropRange)))
        Transform = T.Compose(Transform)

        image = Transform(image)
        GT = Transform(GT)

        ShiftRange_left = random.randint(0,20)
        ShiftRange_upper = random.randint(0,20)
        ShiftRange_right = image.size[0] - random.randint(0,20)
        ShiftRange_lower = image.size[1] - random.randint(0,20)
        image = image.crop(box=(ShiftRange_left,ShiftRange_upper,ShiftRange_right,ShiftRange_lower))
        GT = GT.crop(box=(ShiftRange_left,ShiftRange_upper,ShiftRange_right,ShiftRange_lower))

        if random.random() < 0.5:
            image = F.hflip(image)
            GT = F.hflip(GT)

        Transform =[]


    Transform.append(T.Resize((hw[1], hw[0])))
    #Transform.append(T.ToTensor())
    Transform = T.Compose(Transform)

    image = Transform(image)
    GT = Transform(GT)

#     Norm_ = T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#     image = Norm_(image)

    return image, GT, hw, p_transform

def convertImage(raw):
    img = np.array(raw)
    img[img>255] = 255
    mean = np.mean(img)
    min_img = np.min(img)
    if min_img<-1500:
        min_img = min_img*(1.0/2.0)
    print(mean, min_img)
    
    img = img - mean
    img = img*1.0 + mean*(mean/min_img)*1.1#修对比度和亮度
    img = img/np.max(img)
    #img[img>255] = 255
    img[img<0] = 0
    return img

def removeImage(dataset, dataset_mask, loc):
    tictoc = 0
    rm_mkdir(dataset)
    rm_mkdir(dataset_mask)
    for i in list_Paths:
        list_PathClasses = os.listdir(Paths+i+'/')
        for j in list_PathClasses:
            list_Dcmfiles = os.listdir(Paths+i+'/'+j+'/')
            negative = 0
            for m in list_Dcmfiles:
                tictoc+=1
                
                if '_mask' in m:
                    tmp_img = Image.open(Paths+i+'/'+j+'/'+m)
                    tmp_img = np.array(tmp_img)
                    
                    if np.max(tmp_img)>0:
                        
                        image = sitk.ReadImage(Paths+i+'/'+j+'/'+m[:-9]+'.dcm')
                        image_array = sitk.GetArrayFromImage(image)
                        dcm_image = image_array[0,:,:]
                        convertImg = convertImage(dcm_image)
                        imgRevert = np.repeat(np.expand_dims(convertImg[loc[0]:loc[1],loc[2]:loc[3]],
                                                             axis=2), 3, axis=2)
                        scipy.misc.imsave(dataset+i+'_'+j[0]+'_'+m[:-9]+'.png', imgRevert)
                        
                        tmp_maskar = tmp_img[loc[0]:loc[1],loc[2]:loc[3]]
                        tmp_maskar = np.repeat(np.expand_dims(tmp_maskar,
                                                             axis=2), 3, axis=2)
                        scipy.misc.imsave(dataset_mask+i+'_'+j[0]+'_'+m, tmp_maskar)
                    else:
                        ### 添加負例樣本
                        if random.random() > 0.0:
                            negative +=1
                    
                            image = sitk.ReadImage(Paths+i+'/'+j+'/'+m[:-9]+'.dcm')
                            image_array = sitk.GetArrayFromImage(image)
                            dcm_image = image_array[0,:,:]
                            convertImg = convertImage(dcm_image)
                            imgRevert = np.repeat(np.expand_dims(convertImg[loc[0]:loc[1],loc[2]:loc[3]],
                                                                 axis=2), 3, axis=2)
                            scipy.misc.imsave(dataset+i+'_'+j[0]+'_'+m[:-9]+'.png', imgRevert)


                            tmp_maskar = tmp_img[loc[0]:loc[1],loc[2]:loc[3]]
                            tmp_maskar = np.repeat(np.expand_dims(tmp_maskar,
                                                                 axis=2), 3, axis=2)
                            scipy.misc.imsave(dataset_mask+i+'_'+j[0]+'_'+m, tmp_maskar)
            
            print('patient is {}, negative samples are {}'.format(j, negative))


In [ ]:
removeImage(Dataset, Dataset_mask, Loc)

#### Generate the annotations.json for mask_rcnn

In [ ]:
Paths_mask_val = './ISIC/dataset/validcancer_true_GT/'
#Paths_mask_val = './ISIC/dataset/validcancer_GT/'
Paths_mask_test = './ISIC/dataset/testcancer_true_GT/'
#Paths_mask_test = './ISIC/dataset/testcancer_GT/'
Paths_mask_train = './ISIC/dataset/traincancer_true_GT/'
#Paths_mask_train = './ISIC/dataset/traincancer_GT/'

In [ ]:
def mask2json(paths):
    tmp_json = {}
    MaskImgs_list = os.listdir(paths)
    for i in MaskImgs_list:
        tmp_img = Image.open(paths+i).convert('L')
        tmp_img = np.array(tmp_img)
        query = np.where(tmp_img!=0)

        tmp_json[i+str(random.randint(1,10000))] = {'filename' : i[:-9]+'.png',
                                                    'regions' : {'0' : {'shape_attributes' :
                                                                        {'name' : 'ploygon', 
                                                                         'all_points_x' : query[1].tolist(),
                                                                         'all_points_y' : query[0].tolist()}}}}
    return tmp_json

def mask2json_dataaugmentation(paths, maskrcnn_path):
    tmp_json = {}
    MaskImgs_list = os.listdir(paths)
    for i in MaskImgs_list:
        tmp_img = Image.open(paths+i).convert('L')
        tmp_img = np.array(tmp_img)
        query = np.where(tmp_img!=0)

        tmp_json[i+str(random.randint(1,100))] = {'filename' : i[:-9]+'.png',
                                                    'regions' : {'0' : {'shape_attributes' :
                                                                        {'name' : 'ploygon', 
                                                                         'all_points_x' : query[1].tolist(),
                                                                         'all_points_y' : query[0].tolist()}}}}
        
        tmp_au_raw, tmp_au_raw_gt, _, tmp_prob = data_augmentation(paths[:-4]+'/'+i[:-9]+'.png', paths+i)
        if tmp_prob<=0.7:
            random_num = 'au'
            tmp_au_raw.save(maskrcnn_path+i[:-9]+'_'+random_num+'.png')
            tmp_au_raw_gt = np.array(tmp_au_raw_gt)
            query = np.where(tmp_au_raw_gt!=0)
            tmp_json[i+random_num] = {'filename' : i[:-9]+'_'+random_num+'.png',
                                                    'regions' : {'0' : {'shape_attributes' :
                                                                        {'name' : 'ploygon', 
                                                                         'all_points_x' : query[1].tolist(),
                                                                         'all_points_y' : query[0].tolist()}}}}
            #tmp_au_raw
    return tmp_json

# val
json_val = mask2json(Paths_mask_val)
rm_mkdir('./ISIC/Mask_RCNN-master/dataset/val/')
json.dump(json_val, open('./ISIC/Mask_RCNN-master/dataset/val/via_region_data.json', 'w'))
#! cp -r ./ISIC/dataset/validcancer/* ./ISIC/Mask_RCNN-master/dataset/val/
! cp -r ./ISIC/dataset/validcancer_true/* ./ISIC/Mask_RCNN-master/dataset/val/
# test
json_test = mask2json(Paths_mask_test)
rm_mkdir('./ISIC/Mask_RCNN-master/dataset/test/')
json.dump(json_test, open('./ISIC/Mask_RCNN-master/dataset/test/via_region_data.json', 'w'))
! cp -r ./ISIC/dataset/testcancer/* ./ISIC/Mask_RCNN-master/dataset/test/

# train
mask_rcnn_trainpath = './ISIC/Mask_RCNN-master/dataset/train/'
rm_mkdir(mask_rcnn_trainpath)
json_train = mask2json_dataaugmentation(Paths_mask_train, mask_rcnn_trainpath)
json.dump(json_train, open('./ISIC/Mask_RCNN-master/dataset/train/via_region_data.json', 'w'))
! cp -r ./ISIC/dataset/traincancer/* ./ISIC/Mask_RCNN-master/dataset/train/


In [ ]:
! cp -r ./ISIC/dataset/validcancer_true/* ./ISIC/Mask_RCNN-master/dataset/val/
! cp -r ./ISIC/dataset/testcancer_true/* ./ISIC/Mask_RCNN-master/dataset/test/
! cp -r ./ISIC/dataset/traincancer_true/* ./ISIC/Mask_RCNN-master/dataset/train/